# Advanced Topics: Additional Filtering

The filtering examples we've shown to this point have been pretty simple, either comparisons between columns or fixed values, or set filter functions like `isin` and `notin`. 

Ibis supports a number of richer analytical filters that can involve one or more of:

- Aggregates computed from the same or other tables
- Conditional aggregates (in SQL-speak these are similar to "correlated subqueries")
- "Existence" set filters (equivalent to the SQL `EXISTS` and `NOT EXISTS` keywords)

## Setup

In [ ]:
import ibis
import os
hdfs_port = os.environ.get('IBIS_WEBHDFS_PORT', 50070)
hdfs = ibis.hdfs_connect(host='quickstart.cloudera', port=hdfs_port)
con = ibis.impala.connect(host='quickstart.cloudera', database='ibis_testing',
                          hdfs_client=hdfs)
ibis.options.interactive = True

## Using scalar aggregates in filters

In [ ]:
table = con.table('functional_alltypes')
table.limit(5)

We could always compute some aggregate value from the table and use that in another expression, or we can use a data-derived aggregate in the filter. Take the average of a column for example:

In [ ]:
table.double_col.mean()

You can use this expression as a substitute for a scalar value in a filter, and the execution engine will combine everything into a single query rather than having to access Impala multiple times:

In [ ]:
cond = table.bigint_col > table.double_col.mean()
expr = table[cond & table.bool_col].limit(5)
expr

## Conditional aggregates


Suppose that we wish to filter using an aggregate computed conditional on some other expressions holding true. Using the TPC-H datasets, suppose that we want to filter customers based on the following criteria: Orders such that their amount exceeds the average amount for their sales region over the whole dataset. This can be computed any numbers of ways (such as joining auxiliary tables and filtering post-join)

Again, from prior examples, here are the joined up tables with all the customer data:

In [ ]:
region = con.table('tpch_region')
nation = con.table('tpch_nation')
customer = con.table('tpch_customer')
orders = con.table('tpch_orders')

fields_of_interest = [customer,
                      region.r_name.name('region'), 
                      orders.o_totalprice,
                      orders.o_orderdate.cast('timestamp').name('odate')]

tpch = (region.join(nation, region.r_regionkey == nation.n_regionkey)
        .join(customer, customer.c_nationkey == nation.n_nationkey)
        .join(orders, orders.o_custkey == customer.c_custkey)
        [fields_of_interest])

tpch.limit(5)

In this particular case, filtering based on the conditional average `o_totalprice` by region requires creating a table view (similar to the self-join examples from earlier) that can be treated as a distinct table entity in the expression. This would **not** be required if we were computing a conditional statistic from some other table. So this is a little more complicated than some other cases would be:

In [ ]:
t2 = tpch.view()
conditional_avg = t2[(t2.region == tpch.region)].o_totalprice.mean()

Once you've done this, you can use the conditional average in a filter expression

In [ ]:
amount_filter = tpch.o_totalprice > conditional_avg
tpch[amount_filter].limit(10)

By looking at the table sizes before and after applying the filter you can see the relative size of the subset taken. 

In [ ]:
tpch.count()

In [ ]:
tpch[amount_filter].count()

Or even group by year and compare before and after:

In [ ]:
tpch.schema()

In [ ]:
year = tpch.odate.year().name('year')

pre_sizes = tpch.group_by(year).size()
post_sizes = tpch[amount_filter].group_by(year).size().view()

percent = ((post_sizes['count'] / pre_sizes['count'].cast('double'))
           .name('fraction'))

expr = (pre_sizes.join(post_sizes, pre_sizes.year == post_sizes.year)
        [pre_sizes.year, 
         pre_sizes['count'].name('pre_count'),
         post_sizes['count'].name('post_count'),
         percent])
expr

## "Existence" filters


Some filtering involves checking for the existence of a particular value in a column of another table, or amount the results of some value expression. This is common in many-to-many relationships, and can be performed in numerous different ways, but it's nice to be able to express it with a single concise statement and let Ibis compute it optimally.

Here's some examples:

- Filter down to customers having at least one open order
- Find customers having no open orders with 1-URGENT status
- Find stores (in the stores table) having the same name as a vendor (in the vendors table).

We'll go ahead and solve the first couple of these problems using the TPC-H tables to illustrate the API:

In [ ]:
customer = con.table('tpch_customer')
orders = con.table('tpch_orders')

In [ ]:
orders.limit(5)

We introduce the `any` reduction:

In [ ]:
has_open_orders = ((orders.o_orderstatus == 'O') & 
                   (customer.c_custkey == orders.o_custkey)).any()

This is now a valid filter:

In [ ]:
customer[has_open_orders].limit(10)

For the second example, in which we want to find customers not having any open urgent orders, we write down the condition that they _do_ have some first:

In [ ]:
has_open_urgent_orders = ((orders.o_orderstatus == 'O') & 
                          (orders.o_orderpriority == '1-URGENT') & 
                          (customer.c_custkey == orders.o_custkey)).any()

Now, we can negate this condition and use it as a filter:

In [ ]:
customer[-has_open_urgent_orders].count()

In this case, it is true that `customer.c_custkey` has no duplicate values, but that need not be the case. There could be multiple copies of any given value in either table column being compared, and the behavior will be the same (existence or non-existence is verified).